In [2]:
%env CUDA_VISIBLE_DEVICES=0
%env TMPDIR=/raid/users/ryan_cheng/tmp
import os
import glob
import re
import json
import random
import time
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

np.random.seed(0)

try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    # ../.. for notebook, .. for script
    client = OpenAI(api_key=f.read().rstrip('\n'))

vllm_models = [
    'meta-llama/Llama-2-13b-hf', 'Llama-2-13b-hf', 'Llama13b', 'llama13b',
    'mistralai/Mistral-7B-v0.1', 'Mistral', 'mistral',
    'mistralai/Mixtral-8x7B-v0.1', 'Mixtral', 'mixtral',
    'meta-llama/Llama-2-70b-hf', 'Llama-2-70b-hf', 'Llama70b', 'llama70b',
    'meta-llama/Meta-Llama-3-70B-Instruct', 'Llama-3-70B-Instruct', 'meta-llama/Llama-3.1-70B-Instruct', 'meta-llama/Llama-3.1-8B-Instruct',
    'meta-llama/Meta-Llama-3-8B-Instruct', 'Llama-3-8B-Instruct',
    'mistralai/Mistral-7B-Instruct-v0.1', 'mistral-instruct',
    'google/gemma-7b', 'gemma'
]

# run 'ray start --head --num-gpus <NUM>' in bash first!
def setup_llm(model_name, config):
    if model_name not in vllm_models:
        return None, None
    if config['gpus'] > 1:
        ray.init(ignore_reinit_error=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm = LLM(model=model_name, tensor_parallel_size=config['gpus'], download_dir='/raid/users/ryan_cheng/models', gpu_memory_utilization=0.95, max_model_len=41632)
    return llm, tokenizer

def completion_create_helper(model_name, config, prompt, llm=None):
    # limit prompt in all cases
    if model_name not in vllm_models:
        # for some reason vLLM models simply repeat this last statement if present
        prompt += " Limit your answer to three sentences or less!"

    ret = '' # return the output ret at the end and use to calculate cost

    if model_name == "gpt-3.5-turbo-instruct":
        ret = client.completions.create(
                model="gpt-3.5-turbo-instruct",
                prompt=prompt,
                temperature=0.8,
                max_tokens=config['max_tokens'],
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
        )
        ret = ret.choices[0].text.strip()
        #find_line = ret.find("\n")
        #if find_line != -1:
            #ret = ret[:find_line]
    elif model_name in ["gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-4-turbo", "gpt-4o", "gpt-4o-mini"]:
        ret = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "system", "content": prompt}],
            max_tokens=config['max_tokens']
        )
        ret = ret.choices[-1].message.content

    elif model_name in vllm_models and llm:
        sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=config['max_tokens'])
        output = llm.generate([prompt], sampling_params)
        ret = output[0].outputs[0].text
    else:
        raise NotImplementedError

    # if config['model'] not in vllm_models:
    #     running_cost_for_iteration += api_cost(prompt=prompt, answer=ret, model=config['model'])
    #     if config['verbose']:
    #         print(f"RUNNING COST FOR ITERATION SO FAR: {running_cost_for_iteration}")
    
    return ret

def completion_create(model_name, config, prompt, llm=None, keep_trying=True):
    try:
        return completion_create_helper(model_name, config, prompt, llm)
    except (openai.APIError, openai.OpenAIError) as e:
        # print("ERROR", e)
        # print("sleeping for 10 seconds.")
        time.sleep(10)
        if keep_trying:
            return completion_create(model_name, config, prompt, llm, keep_trying)
        else:
            return None
            
def write_json(write_path, json_dict):
    if not os.path.exists(write_path):
        with open(write_path, 'w') as f:
            json.dump([], f)
    with open(write_path, 'r') as f:
        evaluations = json.load(f)
    evaluations.append(json_dict)
    with open(write_path, 'w') as f:
        json.dump(evaluations, f, indent=4)

env: CUDA_VISIBLE_DEVICES=0
env: TMPDIR=/raid/users/ryan_cheng/tmp
INFO 04-03 02:48:15 __init__.py:190] Automatically detected platform cuda.


In [3]:
config_gpt4_mini = {
    'agent1_model': 'gpt-4o-mini',
    'agent2_model': 'gpt-4o-mini',
    'eval_model': 'gpt-4o-mini',
    'agent1_role': "\nPerson 1: ",
    'agent2_role': "\nPerson 2: ",
    #'generic_prompt': generic, 
    #'agent1_specific': agent_1_specific + do_not_repeat_text, 
    #'agent2_specific': agent_2_specific + do_not_repeat_text,
    #'eval_prompts': eval_prompts,
    'iterations': 1, # not used
    'verbose': True, # not used
    'write': True,
    'convo_length_limit': 10,
    'max_tokens': 256,
    'gpus': 2,
    'task_name': 'Persona Chat',
}

In [7]:
def generate_conversation(p1, p2, config, pturn=1, write=False, write_path='data/persona/persona_chat.json'):
    conv_dict = {
        "task_name": config['task_name'],
        "P1": p1,
        "P2": p2,
        "conversation": [],
        "pturn": pturn # beginning person (1 or 2)
        }
    round_num = 0
    while round_num < config['convo_length_limit']:
        if pturn == 1:
            prompt = "You are P1, and you are having a conversation with P2. Your backstory is:\n" + p1 + "\n" + "So far, the conversation is as below, and it is your turn to speak next.\n" + ("".join(conv_dict["conversation"]) if len(conv_dict["conversation"]) != 0 else "[You are starting the conversation.]") + "P1: "
            pturn = 2
            conv_dict["conversation"].append("P1: " + completion_create(config['agent1_model'], config, prompt) + "\n")
        else:
            prompt = "You are P2, and you are having a conversation with P1. Your backstory is:\n\n" + p2 + "\n\n" + "So far, the conversation is as below, and it is your turn to speak next.\n" + ("".join(conv_dict["conversation"]) if len(conv_dict["conversation"]) != 0 else "[You are starting the conversation.]") + "P2: "
            pturn = 1     
            conv_dict["conversation"].append("P2: " + completion_create(config['agent2_model'], config, prompt) + "\n")
        round_num += 1

    conv_dict["rounds"] = round_num
    if write:
        write_json(write_path, conv_dict)
    return conv_dict


# Persona Chat Generation

In [4]:
persona_chat = pd.read_csv('./data/persona/personality.csv')
persona_chat.head()

,Unnamed: 0,Persona,chat
0,0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."
1,1,my mom is my best friend. i have four sisters...,"hi , how are you doing today ?\ni am spending ..."
2,2,i had a gig at local theater last night. i wo...,"we all live in a yellow submarine , a yellow s..."
3,3,i am very athletic. i wear contacts. i have b...,hi ! i work as a gourmet cook .\ni do not like...
4,4,i am primarily a meat eater. i am a guitar pl...,how are you doing today\nwhat do you do for ca...


In [5]:
len(persona_chat)

8939

In [17]:
print(persona_chat.iloc[0]["Persona"])

 i like to remodel homes. i like to go hunting. i like to shoot a bow. my favorite holiday is halloween.


In [42]:
print(persona_chat.iloc[0]["chat"])

hi , how are you doing ? i am getting ready to do some cheetah chasing to stay in shape .
you must be very fast . hunting is one of my favorite hobbies .
i am ! for my hobby i like to do canning or some whittling .
i also remodel homes when i am not out bow hunting .
that is neat . when i was in high school i placed 6th in 100m dash !
that is awesome . do you have a favorite season or time of year ?
i do not . but i do have a favorite meat since that is all i eat exclusively .
what is your favorite meat to eat ?
i would have to say its prime rib . do you have any favorite foods ?
i like chicken or macaroni and cheese .
do you have anything planned for today ? i think i am going to do some canning .
i am going to watch football . what are you canning ?
i think i will can some jam . do you also play footfall for fun ?
if i have time outside of hunting and remodeling homes . which is not much !



In [45]:
random_indices = np.array([np.random.choice(len(persona_chat), size=2, replace=False) for _ in range(10)])
random_indices # pairs of indices of Persona to  use in conversation

array([[7710, 4426],
       [5525, 2032],
       [ 574, 6234],
       [2303,  263],
       [3890,  877],
       [8913, 6509],
       [7400, 2988],
       [6578, 4878],
       [5014, 8746],
       [8254, 5155]])

In [25]:
persona_chat[persona_chat["chat"] == persona_chat.iloc[0]["chat"]]

,Unnamed: 0,Persona,chat
0,0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."


In [76]:
"".join(["a", "b"])

'ab'

In [81]:
conversations = []
for p1i, p2i in tqdm(random_indices):
    conversations.append(generate_conversation(persona_chat.iloc[p1i]["Persona"], persona_chat.iloc[p2i]["Persona"], config_gpt4_mini, write=True))

100%|██████████| 10/10 [01:44<00:00, 10.50s/it]


In [82]:
conversations


[{'task_name': 'Persona Chat',
  'P1': ' i like to cook. i have lived in several different states. i worked as a nurse for many years. my husband was a salesman. i have a cat named kj.',
  'P2': ' i like fruit. my favorite fruit is apple. i don t like to eat meat. i like to eat the skin of the apple. i like green apples.',
  'conversation': ["P1: P1: Hey! What’s your favorite meal to cook? I love trying out new recipes, especially since I've lived in so many different states and picked up various culinary influences along the way.\n",
   "P2: I don't really cook meat, so I often make dishes that highlight fruits and vegetables. My favorite is a fresh apple salad with green apples, nuts, and a light dressing. It's simple, delicious, and lets the apple's flavor shine!\n",
   'P1: That sounds refreshing! I love incorporating seasonal fruits and veggies into my dishes too. Is there a specific time of year when you enjoy making your apple salad the most?\n',
   "P2: I love making my apple s

# Synthetic Persona Chat

In [4]:
syn_persona_chat = pd.read_csv('./data/synthetic-persona/New-Persona-New-Conversations.csv')
syn_persona_chat.head()

,user 1 personas,user 2 personas,Best Generated Conversation
0,I'm moving to a new city to pursue my culinary...,"I am comfortable with the weather, and enjoy s...","User 1: Hi! How are you?\nUser 2: Good, thanks..."
1,I used my artistic skills to teach children ho...,I like to run and do yoga in my spare time.\nM...,User 1: Hey how are you doing?\nUser 2: Doing ...
2,"I enjoy spending time with my pets, and I love...",I am close to my mother and love to spend time...,"User 1: Hello there, how are you doing today?\..."
3,"My cupcake business keeps me busy, but I also ...",I collect memories and seashells from my trave...,User 1: hi there!\nUser 2: Hey! How's your day...
4,The weather is perfect for cheering!\ni listen...,I had to move across country for my job.\nI go...,User 1: How is your day going?\nUser 2: Great!...


In [21]:
syn_persona_chat.iloc[0]["Best Generated Conversation"]

"User 1: Hi! How are you?\nUser 2: Good, thanks for asking! How about yourself?\nUser 1: I'm doing pretty well.  I'm excited to be moving to a new city soon!\nUser 2: Oh that's awesome! What city are you moving to?\nUser 1: I'm moving to Portland, Oregon.  I'm going to be pursuing my culinary dreams there.\nUser 2: That sounds amazing! I love Portland.  I'm originally from there.\nUser 1: Really? That's so cool!  Do you still live there?\nUser 2: No, I moved away a few years ago.  But I still visit my family there often.\nUser 1: That's great.  Do you have any favorite places to visit in Portland?\nUser 2: Yes, I love going to Powell's Books.  It's the largest independent bookstore in the world.\nUser 1: That sounds amazing!  I've never been to Powell's Books before.  I'm definitely going to have to check it out.\nUser 2: You definitely should!  It's a really cool place.\nUser 1: Thanks for the recommendation!  I'm sure I'll love it.\nUser 2: No problem!  I'm glad I could help.\n"

In [6]:
random_indices = np.array([np.random.choice(len(syn_persona_chat), size=2, replace=False) for _ in range(10)])
random_indices # pairs of indices of Persona to  use in conversation

array([[ 9263,  4795],
       [ 1970,  6026],
       [  523,  2596],
       [ 9094,   194],
       [ 9967,  6999],
       [ 3452,  8207],
       [ 1425,  7914],
       [ 6028,  6878],
       [ 1226, 10490],
       [ 7645,  7320]])

In [ ]:
# Use generated conversations
conversations = []
for p1i, p2i in tqdm(random_indices):
    conversations.append(generate_conversation(syn_persona_chat.iloc[p1i]["user 1 personas"], syn_persona_chat.iloc[p2i]["user 2 personas"], config_gpt4_mini, write=True, write_path='data/persona/syn_persona_chat.json'))

100%|██████████| 10/10 [01:49<00:00, 10.91s/it]


In [11]:
def syn_conversation_to_list(conversation: str) -> list[str]:
    pattern = r"(User 1:|User 2:)(.*?)(?=(User 1:|User 2:|$))"
    matches = re.findall(pattern, conversation, re.DOTALL)

    # Combine consecutive entries with the same speaker
    combined_entries = []
    for speaker, message, _ in matches:
        if combined_entries and combined_entries[-1].startswith(speaker):
            combined_entries[-1] += " " + message.strip()
        else:
            combined_entries.append(f"{speaker.strip()} {message.strip()}")

    return combined_entries

In [12]:
syn_conversation_to_list(syn_persona_chat.iloc[0]["Best Generated Conversation"])

['User 1: Hi! How are you?',
 'User 2: Good, thanks for asking! How about yourself?',
 "User 1: I'm doing pretty well.  I'm excited to be moving to a new city soon!",
 "User 2: Oh that's awesome! What city are you moving to?",
 "User 1: I'm moving to Portland, Oregon.  I'm going to be pursuing my culinary dreams there.",
 "User 2: That sounds amazing! I love Portland.  I'm originally from there.",
 "User 1: Really? That's so cool!  Do you still live there?",
 'User 2: No, I moved away a few years ago.  But I still visit my family there often.',
 "User 1: That's great.  Do you have any favorite places to visit in Portland?",
 "User 2: Yes, I love going to Powell's Books.  It's the largest independent bookstore in the world.",
 "User 1: That sounds amazing!  I've never been to Powell's Books before.  I'm definitely going to have to check it out.",
 "User 2: You definitely should!  It's a really cool place.",
 "User 1: Thanks for the recommendation!  I'm sure I'll love it.",
 "User 2: No 

In [23]:
# Use conversations from dataset instead, taking the index of the conversation to be the first of the pair of random indices
orig_conversations = []
for p1i, p2i in tqdm(random_indices):
    conv = syn_persona_chat.iloc[p1i]
    conv_dict = {
        "task_name": 'Synthetic Persona Chat',
        "P1": conv['user 1 personas'],
        "P2": conv['user 2 personas'],
        "conversation": syn_conversation_to_list(conv["Best Generated Conversation"]),
        "pturn": 1 # beginning person (this dataset always starts with User 1)
        }
    orig_conversations.append(conv_dict)
    

100%|██████████| 10/10 [00:00<00:00, 12554.04it/s]


In [ ]:
for conversation in orig_conversations:
    eval_prompt_consistency(conversation, config_gpt4_mini)
    eval_all_line_consistency(conversation, config_gpt4_mini)
    eval_prev_line_consistency(conversation, config_gpt4_mini)

with open('data/persona/syn_persona_chat.json', 'w') as f:
        json.dump(orig_conversations, f, indent=4)

In [36]:
with open('data/persona/syn_persona_chat.json', 'w') as f:
        json.dump(orig_conversations, f, indent=4)

# Consistency Evaluation

In [28]:
# (1) Takes in dialog, takes in base prompt, checks inconsistencies with base prompt for each line and output

def eval_prompt_consistency(conv_dict, config):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite

    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        if pturn == 1:
            prompt = "For the following line spoken by P1, answer YES if the line contradicts the given backstory of P1, and answer NO if the line does not contradict the provided backstory of P1. P1's backstory is:\n" + conv_dict["P1"] + "\n P1 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts the given backstory of P1, and answer NO if the line does not contradict the provided backstory of P1, followed by 1 sentence of reasoning.\n\n"
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        else:
            prompt = "For the following line spoken by P2, answer YES if the line contradicts the given backstory of P2, and answer NO if the line does not contradict the provided backstory of P2. P2's backstory is:\n" + conv_dict["P2"] + "\n P2 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts the given backstory of P2, and answer NO if the line does not contradict the provided backstory of P2, followed by 1 sentence of reasoning.\n\n"
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P2_prompt_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1

    conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    conv_dict['P2_prompt_consistency_score'] /= p2_utterances

In [33]:
# (2) Takes in dialog, takes in base prompt, checks inconsistencies with every line henceforth 

def eval_all_line_consistency(conv_dict, config):
    conv_dict['eval_all_line_consistency'] = []
    conv_dict['P1_all_line_consistency_score'] = 0
    conv_dict['P2_all_line_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for i, line in enumerate(conv_dict["conversation"]):
        if pturn == 1:
            # old prompt (checks backstory as well)
            #prompt = "For the following line spoken by P1, answer YES if the line contradicts any line stated by P1 or P1's provided background, and answer NO if the line does not contradict any line in the provided conversation history of P1 and P1's provided background. P1 has the following backstory:\n" + conv_dict["P1"] + "\nP1 had the following conversation with P2:\n" + "".join(conv_dict["conversation"]) + "\n P1 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts any line stated by P1 throughout the course of the conversation or P1's provided background, and answer NO if the line does not contradict any line in the provided conversation history of P1 and P1's provided background, followed by 1 sentence of reasoning.\n\n"
            prompt = "For the following line spoken by P1, answer YES if the line contradicts any line stated by P1, and answer NO if the line does not contradict any line in the provided conversation history of P1. \nP1 had the following conversation with P2:\n" + "".join(conv_dict["conversation"]) + "\n P1 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts any line stated by P1 throughout the course of the conversation, and answer NO if the line does not contradict any line in the provided conversation history of P1, followed by 1 sentence of reasoning.\n\n"
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_all_line_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P1_all_line_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        else:
            #prompt = "For the following line spoken by P2, answer YES if the line contradicts any line stated by P2 or P2's provided background, and answer NO if the line does not contradict any line in the provided conversation history of P2 and P2's provided background. P2 has the following backstory:\n" + conv_dict["P2"] + "\nP2 had the following conversation with P1:\n" + "".join(conv_dict["conversation"]) + "\n P2 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts any line stated by P2 throughout the course of the conversation or P2's provided background, and answer NO if the line does not contradict any line in the provided conversation history of P2 and P2's provided background, followed by 1 sentence of reasoning.\n\n"
            prompt = "For the following line spoken by P2, answer YES if the line contradicts any line stated by P2, and answer NO if the line does not contradict any line in the provided conversation history of P2. \nP2 had the following conversation with P1:\n" + "".join(conv_dict["conversation"]) + "\n P2 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts any line stated by P2 throughout the course of the conversation, and answer NO if the line does not contradict any line in the provided conversation history of P2, followed by 1 sentence of reasoning.\n\n"
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_all_line_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P2_all_line_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1

    conv_dict['P1_all_line_consistency_score'] /= p1_utterances
    conv_dict['P2_all_line_consistency_score'] /= p2_utterances

In [34]:
# (4) Takes in dialog, takes in base prompt, checks for inconsistency with previous line 

def eval_prev_line_consistency(conv_dict, config):
    conv_dict['eval_prev_line_consistency'] = []
    conv_dict['P1_prev_line_consistency_score'] = 0
    conv_dict['P2_prev_line_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for i, line in enumerate(conv_dict["conversation"]):
        if pturn == 1:
            
            #prompt = "For the following line spoken by P1, answer YES if the line contradicts a previous line stated by P1 or P1's provided background, and answer NO if the line does not contradict the provided conversation history of P1 and P1's provided background. P1 has the following backstory:\n" + conv_dict["P1"] + "\nP1 had the following conversation with P2:\n" + "".join(conv_dict["conversation"][:i]) + "\n P1 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts a previous line stated by P1 or P1's provided background, and answer NO if the line does not contradict the provided conversation history of P1 and P1's provided background, followed by 1 sentence of reasoning.\n\n"
            prompt = "For the following line spoken by P1, answer YES if the line contradicts a previous line stated by P1, and answer NO if the line does not contradict the provided conversation history of P1. \nP1 had the following conversation with P2:\n" + "".join(conv_dict["conversation"][:i]) + "\n P1 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts a previous line stated by P1, and answer NO if the line does not contradict the provided conversation history of P1, followed by 1 sentence of reasoning.\n\n"
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prev_line_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P1_prev_line_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        else:

            prompt = "For the following line spoken by P2, answer YES if the line contradicts a previous line stated by P2, and answer NO if the line does not contradict the provided conversation history of P2. \nP2 had the following conversation with P1:\n" + "".join(conv_dict["conversation"][:i]) + "\n P2 spoke the following line: \n" + line + "\n\n Answer YES if the line contradicts a previous line stated by P2, and answer NO if the line does not contradict the provided conversation history of P2, followed by 1 sentence of reasoning.\n\n"
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prev_line_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P2_prev_line_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1

    conv_dict['P1_prev_line_consistency_score'] /= p1_utterances
    conv_dict['P2_prev_line_consistency_score'] /= p2_utterances

In [92]:
for conversation in conversations:
    eval_prompt_consistency(conversation, config_gpt4_mini)
conversations

[{'task_name': 'Persona Chat',
  'P1': ' i like to cook. i have lived in several different states. i worked as a nurse for many years. my husband was a salesman. i have a cat named kj.',
  'P2': ' i like fruit. my favorite fruit is apple. i don t like to eat meat. i like to eat the skin of the apple. i like green apples.',
  'conversation': ["P1: P1: Hey! What’s your favorite meal to cook? I love trying out new recipes, especially since I've lived in so many different states and picked up various culinary influences along the way.\n",
   "P2: I don't really cook meat, so I often make dishes that highlight fruits and vegetables. My favorite is a fresh apple salad with green apples, nuts, and a light dressing. It's simple, delicious, and lets the apple's flavor shine!\n",
   'P1: That sounds refreshing! I love incorporating seasonal fruits and veggies into my dishes too. Is there a specific time of year when you enjoy making your apple salad the most?\n',
   "P2: I love making my apple s

In [93]:
with open('data/persona/persona_chat2.json', 'w') as f:
        json.dump(conversations, f, indent=4)

In [107]:
for conversation in conversations:
    eval_all_line_consistency(conversation, config_gpt4_mini)
conversations

[{'task_name': 'Persona Chat',
  'P1': ' i like to cook. i have lived in several different states. i worked as a nurse for many years. my husband was a salesman. i have a cat named kj.',
  'P2': ' i like fruit. my favorite fruit is apple. i don t like to eat meat. i like to eat the skin of the apple. i like green apples.',
  'conversation': ["P1: P1: Hey! What’s your favorite meal to cook? I love trying out new recipes, especially since I've lived in so many different states and picked up various culinary influences along the way.\n",
   "P2: I don't really cook meat, so I often make dishes that highlight fruits and vegetables. My favorite is a fresh apple salad with green apples, nuts, and a light dressing. It's simple, delicious, and lets the apple's flavor shine!\n",
   'P1: That sounds refreshing! I love incorporating seasonal fruits and veggies into my dishes too. Is there a specific time of year when you enjoy making your apple salad the most?\n',
   "P2: I love making my apple s

In [111]:
for conversation in conversations:
    eval_prev_line_consistency(conversation, config_gpt4_mini)
conversations

[{'task_name': 'Persona Chat',
  'P1': ' i like to cook. i have lived in several different states. i worked as a nurse for many years. my husband was a salesman. i have a cat named kj.',
  'P2': ' i like fruit. my favorite fruit is apple. i don t like to eat meat. i like to eat the skin of the apple. i like green apples.',
  'conversation': ["P1: P1: Hey! What’s your favorite meal to cook? I love trying out new recipes, especially since I've lived in so many different states and picked up various culinary influences along the way.\n",
   "P2: I don't really cook meat, so I often make dishes that highlight fruits and vegetables. My favorite is a fresh apple salad with green apples, nuts, and a light dressing. It's simple, delicious, and lets the apple's flavor shine!\n",
   'P1: That sounds refreshing! I love incorporating seasonal fruits and veggies into my dishes too. Is there a specific time of year when you enjoy making your apple salad the most?\n',
   "P2: I love making my apple s

In [112]:
with open('data/persona/persona_chatf.json', 'w') as f:
        json.dump(conversations, f, indent=4)